1. Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Set working directory

In [4]:
import os
os.chdir('/content/drive/MyDrive/Capstone4')
os.getcwd()

'/content/drive/MyDrive/Capstone4'

3. Unzip Dataset

In [ ]:
!unzip /content/drive/MyDrive/Capstone4/data.zip

Archive:  /content/drive/MyDrive/Capstone4/data.zip
  inflating: data.yaml               
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
   creating: test/
   creating: test/images/
  inflating: test/images/-0378D390-5AE2-4D09-84BB-387E850F5E5A-png_jpg.rf.6bed66dadd7ff57f62ec5ad62ebe37f7.jpg  
  inflating: test/images/-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.221530132a9ec242d5fd9d9a76557eb2.jpg  
  inflating: test/images/-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.650f23d30d8750dbe1257b9e741506a2.jpg  
  inflating: test/images/-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.ccbd26a1031345facc1b539e776da48a.jpg  
  inflating: test/images/-0471C451-8CA5-4CEA-8EFD-3B0F147BF4B7-png_jpg.rf.df127c999b5a2dce6eacc30f6cce7365.jpg  
  inflating: test/images/-0585CFA0-7CD6-45D3-B7EB-1A109425F90D-png_jpg.rf.4b8d2eae8c685b8df05a0d52633a939d.jpg  
  inflating: test/images/-0585CFA0-7CD6-45D3-B7EB-1A109425F90D-png_jpg.rf.736978ed48c6ef022e18b100a5a415e3.jpg 

4. Dataset Checking

  - Karena model akan digunakan untuk vehicle detection, maka akan menggunakan pretrained Model YOLOv12n.
  - Untuk proses fine tune / training YOLO model maka dibutuhkan dataset (train, valid, test, data.yaml). Dimana masing2 folder train, valid, dan test memiliki folder images dan labels.
  - Pengechekan kelengkapan label.txt setiap image.
  - Pengechekan label hanya mengandung class, x_center, y_center, width height
  - Semua nilai ternomalisasi [0,1]


In [7]:
DATASET_PATH = "/content/drive/MyDrive/Capstone4"

NUM_CLASSES   = 3   # jumlah class

image_extensions = [".jpg", ".jpeg", ".png"]

print("Dataset root:", DATASET_PATH)

Dataset root: /content/drive/MyDrive/Capstone4


  - Pendefinisian Fungsi pengecekan label YOLO

In [14]:
import re
import glob
import cv2
import matplotlib.pyplot as plt

def check_label_file(txt_path):
    errors = []

    with open(txt_path, "r") as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]

    if len(lines) == 0:
        errors.append("Label kosong")
        return errors

    for idx, line in enumerate(lines, start=1):
        parts = line.split()

        if len(parts) != 5:
            errors.append(f"Line {idx}: format salah → '{line}'")
            continue

        try:
            cls = int(parts[0])
            vals = list(map(float, parts[1:]))
        except:
            errors.append(f"Line {idx}: bukan angka valid → '{line}'")
            continue

        if cls < 0 or cls >= NUM_CLASSES:
            errors.append(f"Line {idx}: class ID salah: {cls}")

        for j, v in enumerate(vals):
            if not (0 <= v <= 1):
                errors.append(f"Line {idx}: nilai ke-{j+1} ({v}) di luar [0–1]")

    return errors

  - Loop semua gambar dan cek label

In [15]:
splits = ["train", "valid", "test"]

missing_labels = []
label_errors = {}
missing_images = []   # ← Tambahan! Label tanpa gambar


for split in splits:
    image_folder = os.path.join(DATASET_PATH, split, "images")
    label_folder = os.path.join(DATASET_PATH, split, "labels")

    print(f"\nScanning split: {split}")

    images = []
    for ext in image_extensions:
        images.extend(glob.glob(os.path.join(image_folder, "*" + ext)))

    image_basenames = set(os.path.splitext(os.path.basename(x))[0] for x in images)
    label_files = glob.glob(os.path.join(label_folder, "*.txt"))
    label_basenames = set(os.path.splitext(os.path.basename(x))[0] for x in label_files)

    # ------------ Cek gambar tanpa label ------------
    for img_path in images:
        base = os.path.splitext(os.path.basename(img_path))[0]
        txt_path = os.path.join(label_folder, base + ".txt")

        if not os.path.exists(txt_path):
            missing_labels.append(img_path)
            continue

        errs = check_label_file(txt_path)
        if errs:
            label_errors[txt_path] = errs

    # ------------ Cek label tanpa gambar ------------
    for txt_path in label_files:
        base = os.path.splitext(os.path.basename(txt_path))[0]
        img_exists = any(
            os.path.exists(os.path.join(image_folder, base + ext))
            for ext in image_extensions
        )
        if not img_exists:
            missing_images.append(txt_path)


# ================================
# 5. LAPORAN AKHIR
# ================================
print("\n===================================")
print("      HASIL PEMERIKSAAN DATASET    ")
print("===================================\n")

print(f"❌ Gambar tanpa label: {len(missing_labels)}")
print(f"❌ Label tanpa gambar: {len(missing_images)}")
print(f"⚠️ Label bermasalah: {len(label_errors)}")


Scanning split: train

Scanning split: valid

Scanning split: test

      HASIL PEMERIKSAAN DATASET    

❌ Gambar tanpa label: 0
❌ Label tanpa gambar: 0
⚠️ Label bermasalah: 72


  - Mengahpus Data yang tidak valid

In [16]:
# ============================================================
# 6. HAPUS FILE YANG BERMASALAH
# ============================================================

delete = input("\nApakah Anda ingin menghapus file yang tidak berpasangan? (y/n): ")

if delete.lower() == "y":

    # Hapus gambar tanpa label
    for img in missing_labels:
        try:
            os.remove(img)
            print("Dihapus:", img)
        except:
            print("Gagal hapus:", img)

    # Hapus label tanpa gambar
    for txt in missing_images:
        try:
            os.remove(txt)
            print("Dihapus:", txt)
        except:
            print("Gagal hapus:", txt)

    print("\n🔥 Semua file tidak berpasangan berhasil dibersihkan.")
else:
    print("\nTidak ada file yang dihapus.")

print("\nPemeriksaan selesai.")


Apakah Anda ingin menghapus file yang tidak berpasangan? (y/n): y

🔥 Semua file tidak berpasangan berhasil dibersihkan.

Pemeriksaan selesai.


In [17]:
splits = ["train", "valid", "test"]

missing_labels = []
label_errors = {}
missing_images = []   # ← Tambahan! Label tanpa gambar


for split in splits:
    image_folder = os.path.join(DATASET_PATH, split, "images")
    label_folder = os.path.join(DATASET_PATH, split, "labels")

    print(f"\nScanning split: {split}")

    images = []
    for ext in image_extensions:
        images.extend(glob.glob(os.path.join(image_folder, "*" + ext)))

    image_basenames = set(os.path.splitext(os.path.basename(x))[0] for x in images)
    label_files = glob.glob(os.path.join(label_folder, "*.txt"))
    label_basenames = set(os.path.splitext(os.path.basename(x))[0] for x in label_files)

    # ------------ Cek gambar tanpa label ------------
    for img_path in images:
        base = os.path.splitext(os.path.basename(img_path))[0]
        txt_path = os.path.join(label_folder, base + ".txt")

        if not os.path.exists(txt_path):
            missing_labels.append(img_path)
            continue

        errs = check_label_file(txt_path)
        if errs:
            label_errors[txt_path] = errs

    # ------------ Cek label tanpa gambar ------------
    for txt_path in label_files:
        base = os.path.splitext(os.path.basename(txt_path))[0]
        img_exists = any(
            os.path.exists(os.path.join(image_folder, base + ext))
            for ext in image_extensions
        )
        if not img_exists:
            missing_images.append(txt_path)


# ================================
# 5. LAPORAN AKHIR
# ================================
print("\n===================================")
print("      HASIL PEMERIKSAAN DATASET    ")
print("===================================\n")

print(f"❌ Gambar tanpa label: {len(missing_labels)}")
print(f"❌ Label tanpa gambar: {len(missing_images)}")
print(f"⚠️ Label bermasalah: {len(label_errors)}")


Scanning split: train

Scanning split: valid

Scanning split: test

      HASIL PEMERIKSAAN DATASET    

❌ Gambar tanpa label: 0
❌ Label tanpa gambar: 0
⚠️ Label bermasalah: 72


In [18]:
def show_label_errors(label_errors, dataset_path):
    print("===================================")
    print("    MENAMPILKAN LABEL BERMASALAH   ")
    print("===================================\n")

    if len(label_errors) == 0:
        print("Tidak ada label yang bermasalah ✔️")
        return

    for txt_path, errors in label_errors.items():
        print("\n--------------------------------")
        print("LABEL ERROR:", txt_path)
        print("--------------------------------")

        # Tampilkan daftar error
        for e in errors:
            print(" -", e)

        # Cari file gambar terkait
        base = os.path.splitext(os.path.basename(txt_path))[0]

        # Coba cari gambar
        img_path = None
        for ext in [".jpg", ".jpeg", ".png"]:
            p = txt_path.replace("labels", "images").replace(".txt", ext)
            if os.path.exists(p):
                img_path = p
                break

        if img_path is None:
            print("❌ Gambar tidak ditemukan untuk label ini.")
            continue

        # Load gambar
        img = cv2.imread(img_path)
        if img is None:
            print("❌ Gambar tidak bisa dibaca.")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Plot gambar sebelum bounding box
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Gambar terkait: {os.path.basename(img_path)}")
        plt.axis("off")
        plt.show()

        # Tampilkan bounding box jika valid
        print("Menampilkan bounding box (jika masih bisa diproses)...")

        try:
            h, w = img.shape[:2]

            with open(txt_path, "r") as f:
                lines = [l.strip() for l in f.readlines() if l.strip()]

            for line in lines:
                parts = line.split()
                if len(parts) != 5:
                    continue

                cls = int(parts[0])
                x, y, bw, bh = map(float, parts[1:])

                # Konversi YOLO → pixel
                x1 = int((x - bw/2) * w)
                y1 = int((y - bh/2) * h)
                x2 = int((x + bw/2) * w)
                y2 = int((y + bh/2) * h)

                # Draw box
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, str(cls), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            plt.figure(figsize=(8, 6))
            plt.imshow(img)
            plt.title("Visualisasi Bounding Box (jika valid)")
            plt.axis("off")
            plt.show()

        except Exception as e:
            print("⚠️ Tidak bisa divisualisasikan:", e)


In [19]:
show_label_errors(label_errors, DATASET_PATH)

Output hidden; open in https://colab.research.google.com to view.

  - Menghapus image yang memiliki masalah dengan label

In [20]:
def delete_images_with_label_errors(label_errors):
    print("\n=======================================")
    print("   MENGHAPUS GAMBAR DENGAN LABEL ERROR ")
    print("=======================================\n")

    if len(label_errors) == 0:
        print("Tidak ada label bermasalah. Tidak ada file yang dihapus.")
        return

    confirm = input("Hapus semua gambar yang memiliki label bermasalah? (y/n): ")

    if confirm.lower() != "y":
        print("❌ Operasi dibatalkan.")
        return

    deleted_count = 0

    for txt_path in label_errors.keys():
        base = os.path.splitext(os.path.basename(txt_path))[0]

        # cari gambar yang cocok
        img_path = None
        for ext in [".jpg", ".jpeg", ".png"]:
            p = txt_path.replace("labels", "images").replace(".txt", ext)
            if os.path.exists(p):
                img_path = p
                break

        if img_path is not None:
            try:
                os.remove(img_path)
                print("🗑️ Dihapus:", img_path)
                deleted_count += 1
            except:
                print("❌ Gagal menghapus:", img_path)
        else:
            print("⚠️ Tidak ditemukan gambar untuk:", txt_path)

    print(f"\n🔥 Selesai! Total gambar dihapus: {deleted_count}")


In [21]:
delete_images_with_label_errors(label_errors)


   MENGHAPUS GAMBAR DENGAN LABEL ERROR 

Hapus semua gambar yang memiliki label bermasalah? (y/n): y
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-135-_jpg.rf.65470f7753c743e8213cc58ddef1a82f.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-107-_jpg.rf.c7a0e3788a85dd1a1c9fa276ab041c89.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/Screenshot-2025-04-05-162445-Copy_png.rf.32a715225c5e237b2cc5c7b3805f7c45.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-114-_jpg.rf.2219f90ba088fdce523d9db6d4977d72.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-121-_jpg.rf.69221f094533a055b852c6cce26ac3c0.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-107-_jpg.rf.cb46575540b5e01b6f716f41a6ceddb3.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-175-_jpg.rf.47b539f414fb1b4009b7111252ffa430.jpg
🗑️ Dihapus: /content/drive/MyDrive/Capstone4/train/images/bus-11-_jpg.rf.e7b054f4f51d14a9452d6fd

  - List Image Size

In [24]:
def get_min_max_wh(dataset_path, image_extensions=[".jpg", ".jpeg", ".png"]):
    splits = ["train", "valid", "test"]
    widths = []
    heights = []

    print("\n===================================")
    print("  MENGHITUNG NILAI MIN & MAX SIZE  ")
    print("===================================\n")

    for split in splits:
        image_folder = os.path.join(dataset_path, split, "images")

        images = []
        for ext in image_extensions:
            images.extend(glob.glob(os.path.join(image_folder, "*" + ext)))

        for img_path in images:
            img = cv2.imread(img_path)
            if img is None:
                continue

            h, w = img.shape[:2]
            widths.append(w)
            heights.append(h)

    if not widths:
        print("Tidak ada gambar ditemukan.")
        return

    print(f"📌 Width Minimum : {min(widths)}")
    print(f"📌 Width Maksimum: {max(widths)}\n")

    print(f"📌 Height Minimum : {min(heights)}")
    print(f"📌 Height Maksimum: {max(heights)}\n")

    return {
        "min_width": min(widths),
        "max_width": max(widths),
        "min_height": min(heights),
        "max_height": max(heights),
    }


In [25]:
result = get_min_max_wh(DATASET_PATH)


  MENGHITUNG NILAI MIN & MAX SIZE  

📌 Width Minimum : 120
📌 Width Maksimum: 2048

📌 Height Minimum : 88
📌 Height Maksimum: 2048



  - Check Persebaran Data setiap ID class

In [26]:
from collections import Counter

# === Folder root dataset ===
dataset_root = DATASET_PATH
splits = ["train", "valid", "test"]

# === Function untuk memproses satu folder labels ===
def process_label_folder(label_path):
    class_counts = Counter()
    invalid_lines = 0
    skipped_missing = 0

    if not os.path.exists(label_path):
        print(f"❌ Folder tidak ditemukan: {label_path}")
        return class_counts, invalid_lines, skipped_missing, 0

    label_files = [f for f in os.listdir(label_path) if f.endswith(".txt")]

    for file_name in label_files:
        file_path = os.path.join(label_path, file_name)

        if not os.path.exists(file_path):
            skipped_missing += 1
            continue

        try:
            with open(file_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 1 and parts[0].isdigit():
                        class_id = int(parts[0])
                        class_counts[class_id] += 1
                    else:
                        invalid_lines += 1

        except Exception as e:
            print(f"⚠️ Error reading {file_name}: {e}")

    return class_counts, invalid_lines, skipped_missing, len(label_files)


# === PROCESS ALL SPLITS (train, valid, test) ===
results = {}

for split in splits:
    label_path = os.path.join(dataset_root, split, "labels")
    print(f"\n🔍 Memproses folder: {label_path}")
    results[split] = process_label_folder(label_path)

# === Print hasil ===
print("\n==============================")
print("    YOLO LABEL SUMMARY")
print("==============================\n")

total_counts = Counter()

for split in splits:
    class_counts, invalid, skipped, total_files = results[split]

    print(f"=== {split.upper()} SET ===")
    print(f"Total label files : {total_files}")
    print(f"Invalid lines      : {invalid}")
    print(f"Skipped missing    : {skipped}")

    if class_counts:
        print("\nClass ID | Count")
        print("----------------")
        for cls, count in sorted(class_counts.items()):
            print(f"{cls:7} | {count}")
            total_counts[cls] += count
    else:
        print("⚠️ Tidak ada class ditemukan.")

    print("\n------------------------------\n")

# === Print TOTAL SUMMARY ===
print("======= TOTAL DARI SEMUA SPLIT =======")
print("Class ID | Total Count")
print("------------------------")
for cls, count in sorted(total_counts.items()):
    print(f"{cls:7} | {count}")


🔍 Memproses folder: /content/drive/MyDrive/Capstone4/train/labels

🔍 Memproses folder: /content/drive/MyDrive/Capstone4/valid/labels

🔍 Memproses folder: /content/drive/MyDrive/Capstone4/test/labels

    YOLO LABEL SUMMARY

=== TRAIN SET ===
Total label files : 9218
Invalid lines      : 0
Skipped missing    : 0

Class ID | Count
----------------
      0 | 4476
      1 | 10739
      2 | 3279

------------------------------

=== VALID SET ===
Total label files : 287
Invalid lines      : 0
Skipped missing    : 0

Class ID | Count
----------------
      0 | 113
      1 | 279
      2 | 35

------------------------------

=== TEST SET ===
Total label files : 220
Invalid lines      : 0
Skipped missing    : 0

Class ID | Count
----------------
      0 | 109
      1 | 236
      2 | 68

------------------------------

======= TOTAL DARI SEMUA SPLIT =======
Class ID | Total Count
------------------------
      0 | 4698
      1 | 11254
      2 | 3382


In [30]:
from collections import defaultdict, Counter
from PIL import Image

splits = ["train", "valid", "test"]

def process_dataset_split(split_path):
    labels_path = os.path.join(split_path, "labels")
    images_path = os.path.join(split_path, "images")

    class_counts = Counter()
    w_norm_sum = defaultdict(float)
    h_norm_sum = defaultdict(float)

    w_pixel_sum = defaultdict(float)
    h_pixel_sum = defaultdict(float)

    img_w_sum = 0
    img_h_sum = 0
    img_count = 0

    invalid_lines = 0
    skipped_missing = 0

    if not os.path.exists(labels_path):
        print(f"❌ Label folder tidak ditemukan: {labels_path}")
        return None

    label_files = [f for f in os.listdir(labels_path) if f.endswith(".txt")]

    for label_file in label_files:
        label_fp = os.path.join(labels_path, label_file)

        # Tentukan file gambar
        image_name = label_file.replace(".txt", ".jpg")
        img_fp = os.path.join(images_path, image_name)
        if not os.path.exists(img_fp):
            image_name = label_file.replace(".txt", ".png")
            img_fp = os.path.join(images_path, image_name)

        if not os.path.exists(img_fp):
            skipped_missing += 1
            continue

        # Baca ukuran gambar
        try:
            img = Image.open(img_fp)
            img_w, img_h = img.size
            img_w_sum += img_w
            img_h_sum += img_h
            img_count += 1
        except:
            print(f"⚠️ Tidak bisa membaca gambar: {img_fp}")
            continue

        # Baca label YOLO
        try:
            with open(label_fp, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5 and parts[0].isdigit():
                        cls = int(parts[0])
                        w_norm = float(parts[3])
                        h_norm = float(parts[4])

                        class_counts[cls] += 1

                        # Normalized YOLO data
                        w_norm_sum[cls] += w_norm
                        h_norm_sum[cls] += h_norm

                        # Pixel bounding box size
                        w_pixel_sum[cls] += w_norm * img_w
                        h_pixel_sum[cls] += h_norm * img_h
                    else:
                        invalid_lines += 1
        except Exception as e:
            print(f"⚠️ Error membaca {label_file}: {e}")
            continue

    return {
        "class_counts": class_counts,
        "w_norm_sum": w_norm_sum,
        "h_norm_sum": h_norm_sum,
        "w_pixel_sum": w_pixel_sum,
        "h_pixel_sum": h_pixel_sum,
        "invalid_lines": invalid_lines,
        "skipped_missing": skipped_missing,
        "img_count": img_count,
        "img_w_avg": img_w_sum / img_count if img_count else 0,
        "img_h_avg": img_h_sum / img_count if img_count else 0,
    }


# === PROCESS TRAIN, VALID, TEST ===
results = {}
for split in splits:
    split_path = os.path.join(dataset_root, split)
    print(f"\n🔍 Memproses: {split.upper()}")
    results[split] = process_dataset_split(split_path)

# === PRINT RESULT ===
print("\n======================================")
print("  YOLO BOUNDING BOX & IMAGE STATISTICS")
print("======================================\n")

for split in splits:
    r = results[split]
    if r is None:
        continue

    print(f"\n======== {split.upper()} SET ========")
    print(f"Total images terbaca : {r['img_count']}")
    print(f"Rata2 image size     : {r['img_w_avg']:.1f} x {r['img_h_avg']:.1f}")
    print(f"Invalid lines         : {r['invalid_lines']}")
    print(f"Missing images        : {r['skipped_missing']}")

    print("\nClass | Count | Avg W(norm) | Avg H(norm) | Avg W(px) | Avg H(px)")
    print("------------------------------------------------------------------")

    for cls in sorted(r["class_counts"].keys()):
        count = r["class_counts"][cls]

        avg_w_norm = r["w_norm_sum"][cls] / count
        avg_h_norm = r["h_norm_sum"][cls] / count

        avg_w_px = r["w_pixel_sum"][cls] / count
        avg_h_px = r["h_pixel_sum"][cls] / count

        print(f"{cls:5} | {count:5} | {avg_w_norm:.4f}     | {avg_h_norm:.4f}     | {avg_w_px:.1f}    | {avg_h_px:.1f}")

    print("------------------------------------------------------------------\n")


🔍 Memproses: TRAIN

🔍 Memproses: VALID

🔍 Memproses: TEST

  YOLO BOUNDING BOX & IMAGE STATISTICS


======== TRAIN SET ========
Total images terbaca : 9164
Rata2 image size     : 624.4 x 512.0
Invalid lines         : 0
Missing images        : 54

Class | Count | Avg W(norm) | Avg H(norm) | Avg W(px) | Avg H(px)
------------------------------------------------------------------
    0 |  4392 | 0.3826     | 0.4733     | 215.3    | 199.4
    1 | 10739 | 0.3171     | 0.3177     | 211.0    | 175.6
    2 |  3279 | 0.4937     | 0.4711     | 270.3    | 198.5
------------------------------------------------------------------


======== VALID SET ========
Total images terbaca : 275
Rata2 image size     : 664.4 x 606.8
Invalid lines         : 0
Missing images        : 12

Class | Count | Avg W(norm) | Avg H(norm) | Avg W(px) | Avg H(px)
------------------------------------------------------------------
    0 |    94 | 0.3653     | 0.5248     | 221.3    | 197.4
    1 |   279 | 0.4778     | 0.4547